## CNN-static implementation with Keras & Gensim
- A model with pre-trained vectors from word2vec. 
- All words including the unknown ones that are randomly initialized are kept static and only the other parameters of the model are learned.
- The dataset could be downloaded at: http://ai.stanford.edu/~amaas/data/sentiment/

In [56]:
import os
import numpy as np
import re
import random

from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from multiprocessing import Pool
from nltk.corpus import stopwords

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, concatenate, Input
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras import backend as K
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ModelCheckpoint
from keras.constraints import maxnorm

In [57]:
stop_words = set(stopwords.words('english'))    # english stopwords from nltk

In [58]:
# lists to contain reviews and labels (pos/neg)
review_list = []
labels_list = []

In [59]:
num_instances = 5000   # number of instances to consider. Actual number of whole data instances would be (num_instances * 4)

In [30]:
# reading text files. 
files = os.listdir('aclImdb/train/pos')[:num_instances]
for file in files:
    with open('aclImdb/train/pos/{}'.format(file), 'r', encoding = 'utf-8') as f:
        sentence = [word for word in f.read().split() if word not in stop_words]
        sentence = [word.lower() for word in sentence if re.match('^[a-zA-Z]+', word)]
        f.close()
    review_list.append(sentence)
    labels_list.append(1)

files = os.listdir('aclImdb/train/neg')[:num_instances]
for file in files:
    review = ''
    with open('aclImdb/train/neg/{}'.format(file), 'r', encoding = 'utf-8') as f:
        sentence = [word for word in f.read().split() if word not in stop_words]
        sentence = [word.lower() for word in sentence if re.match('^[a-zA-Z]+', word)]
        f.close()
    review_list.append(sentence)
    labels_list.append(0)

files = os.listdir('aclImdb/test/pos')[:num_instances]
for file in files:
    review = ''
    with open('aclImdb/test/pos/{}'.format(file), 'r', encoding = 'utf-8') as f:
        sentence = [word for word in f.read().split() if word not in stop_words]
        sentence = [word.lower() for word in sentence if re.match('^[a-zA-Z]+', word)]
        f.close()
    review_list.append(sentence)
    labels_list.append(1)

files = os.listdir('aclImdb/test/neg')[:num_instances]
for file in files:
    review = ''
    with open('aclImdb/test/neg/{}'.format(file), 'r', encoding = 'utf-8') as f:
        sentence = [word for word in f.read().split() if word not in stop_words]
        sentence = [word.lower() for word in sentence if re.match('^[a-zA-Z]+', word)]
        f.close()
    review_list.append(sentence)
    labels_list.append(0)

In [31]:
print(len(review_list))
print(len(labels_list))

20000
20000


In [32]:
max_len = len(max(review_list, key = len))    # maximum length of a sentence
print(max_len)

1508


In [33]:
threshold = max_len = 500   # in order to cut out excessively long sentences, define a thresold (i.e., a maximum number of words to be considered)

In [34]:
# if the length of the sentence is longer than the threshold, exclude that sentence
for i in range(len(review_list)):
    if len(review_list[i]) > threshold :
        review_list[i] = None
        labels_list[i] = None

In [35]:
review_list = [rev for rev in review_list if rev is not None] 
labels_list = [rev for rev in labels_list if rev is not None] 

In [36]:
print(len(review_list))
print(len(labels_list))

19815
19815


In [21]:
embed_dim = 100    # assign the dimension of the embedding space

In [22]:
model = Word2Vec(sentences = review_list, size = embed_dim, sg = 1, window = 5, min_count = 1, iter = 10, workers = Pool()._processes)
model.init_sims(replace = True)    

In [41]:
# create a 3-D numpy array to carry X data
X_data = np.zeros((len(review_list), max_len, embed_dim))

In [42]:
for i in range(len(review_list)):
    for j in range(max_len):
        try:
            X_data[i][j] = model[review_list[i][j]]
        except:
            pass   # if the word is not included in the embedding space, assign zero (i.e., zero padding)

In [43]:
X_data = X_data.reshape(X_data.shape[0], X_data.shape[1], X_data.shape[2], 1)    # reshape the data into 4-D shape
print(X_data.shape)

(19815, 500, 100, 1)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(X_data, labels_list, test_size = 0.3, random_state = 777)

In [60]:
# assign the hyperparameters of the model
filter_sizes = [3, 4, 5]
dropout_rate = 0.5
l2_constraint = 3.0

In [66]:
def convolution():
    inn = Input(shape = (max_len, embed_dim, 1))
    convolutions = []
    # we conduct three convolutions & poolings then concatenate them.
    for fs in filter_sizes:
        conv = Conv2D(filters = 100, kernel_size = (fs, embed_dim), strides = 1, padding = "valid")(inn)
        nonlinearity = Activation('relu')(conv)
        maxpool = MaxPooling2D(pool_size = (max_len - fs + 1, 1), padding = "valid")(nonlinearity)
        convolutions.append(maxpool)
    outt = concatenate(convolutions)
    model = Model(input = inn, output = outt)
    
    return model

In [67]:
def cnn_model():
    convolutions = convolution()
    
    model = Sequential()
    model.add(convolutions)
    model.add(Dropout(dropout_rate))
    
    model.add(Flatten())
    model.add(Dense(1, kernel_constraint=maxnorm(l2_constraint), activation = 'sigmoid'))
    
    adam = optimizers.Adam(lr = 0.01)
    model.compile(optimizer = adam, loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [68]:
cnn = cnn_model()
cnn.summary()    # summary of the model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_2 (Model)              (None, 1, 1, 300)         120300    
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 1, 300)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 301       
Total params: 120,601
Trainable params: 120,601
Non-trainable params: 0
_________________________________________________________________


C:\Users\Buomsoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  # Remove the CWD from sys.path while we load stuff.


In [63]:
cnn_model = KerasClassifier(build_fn = cnn_model, epochs = 100, batch_size = 50, verbose = 1)

In [49]:
cnn_model.fit(X_train, y_train)

C:\Users\Buomsoo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/100
13870/13870 [==============================] - 19s - loss: 0.4020 - acc: 0.8125    
Epoch 2/100
13870/13870 [==============================] - 9s - loss: 0.3068 - acc: 0.8684     
Epoch 3/100
13870/13870 [==============================] - 9s - loss: 0.2590 - acc: 0.8934     
Epoch 4/100
13870/13870 [==============================] - 9s - loss: 0.2319 - acc: 0.9043     
Epoch 5/100
13870/13870 [==============================] - 9s - loss: 0.1902 - acc: 0.9232     
Epoch 6/100
13870/13870 [==============================] - 9s - loss: 0.1623 - acc: 0.9358     
Epoch 7/100
13870/13870 [==============================] - 9s - loss: 0.1376 - acc: 0.9469     
Epoch 8/100
13870/13870 [==============================] - 9s - loss: 0.1256 - acc: 0.9540     
Epoch 9/100
13870/13870 [==============================] - 9s - loss: 0.1071 - acc: 0.9629     
Epoch 10/100
13870/13870 [==============================] - 9s - loss: 0.0966 - acc: 0.9650     
Epoch 11/100
13870/13870 [=============

In [52]:
y_pred = cnn_model.predict(X_test)

5850/5945 [============================>.] - ETA: 0s

In [55]:
print(accuracy_score(y_test, y_pred))

0.883095037847
